# Advanced scikitlearn

We have seen in the last document, some advantages of `scikitlearn`. Most notably the seamless integration of parallel processing. I was struggeling a bit with the fact that `scikitlearn` only accepts `numpy` arrays as input and I was missing the `recipes` package which makes initial data transformation in `R` so much easier. Then I stumbled upon `sklearn-pandas` which seamlessly integrates `pandas` with `sklearn` and supports a pipe based workflow, which is a `sklearn` feaure I have not started to explore yet. To my understanding the pipe workflow in `sklearn` is an object-oriented pythonic version of the modelling dataframe concept introduced by the `tidyverse` in `R`. Finally there are packages like `TPOT` and `auto-sklearn` which automate the whole thing.

In general there are a number of projects that use the synthax and structure of scikit learn, a collection of them can be found at

-[http://contrib.scikit-learn.org/imbalanced-learn/stable/index.html](https://github.com/scikit-learn-contrib/scikit-learn-contrib/blob/master/README.md)

-[http://scikit-learn.org/stable/related_projects.html](http://scikit-learn.org/stable/related_projects.html)


# To sparse or not to sparse

In the `python` data world data is considered to be *sparse* or *dense*. Which adresses the number of zeros in a matrix [wiki](https://en.wikipedia.org/wiki/Sparse_matrix). *sparse* means that you have a lot of them while *dense* means the opposite. There is no particular threshold but we should be aware that some data transformatios like dummy encoding make our data more *sparse*. A *sparse* matrix can be stored in a more memory efficient format such similar as a compressed image file and some algorithms can computationally leaverage this format to reduce computing time. [lasso](http://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_dense_vs_sparse_data.html) and [boosting gradient style algorithms](https://medium.com/sbc-group-blog/to-sparse-or-not-to-sparse-691483f87a53) seem to be able to profit from the sparse data format while others [neural nets, knn](https://medium.com/sbc-group-blog/to-sparse-or-not-to-sparse-691483f87a53) do not, and some like [randomForest](https://stackoverflow.com/questions/28384680/scikit-learns-pipeline-a-sparse-matrix-was-passed-but-dense-data-is-required) require the regular dense format and will otherwise raise an error. We can use `SciPy` to transform matrices to a dense format. We can measure the sparcity ratio as follows


In [14]:
def sparsity_ratio(X):
    return 1.0 - np.count_nonzero(X) / float(X.shape[0] * X.shape[1])




## `sklearn-pandas`

- [github](https://github.com/scikit-learn-contrib/sklearn-pandas)

Core of this package is the `DataFrameMapper` class which maps scikit learn Transformer classes to specific columns of a dataframe and outputs either a numpy array or dataframe.

Additionally it provides a `CategoricalImputer` which accepts categorical data, which I had to write myself before in the last document.

In [15]:
import seaborn as sns
import numpy as np
import pandas as pd
import sklearn

from sklearn_pandas import DataFrameMapper, CategoricalImputer, gen_features

df = sns.load_dataset('titanic')

X = df.copy().drop(['alive','survived'], axis = 'columns')
y = df.survived

# we need to set up transformations for numerical and categorical columns
col_categorical = list( X.select_dtypes(exclude=np.number) )
col_numerical   = list( X.select_dtypes(include=np.number) )

#we need to convert to list of lists
col_categorical = [ [x] for x in col_categorical ]
col_numerical   = [ [x] for x in col_numerical ]

# we have two ways of passing the classes as a simple list or as list of dicts if we need to pass
# arguments as well
classes_categorical = [ CategoricalImputer, sklearn.preprocessing.LabelBinarizer ]
classes_numerical = [ {'class':sklearn.preprocessing.Imputer, 'strategy' : 'median'}
                    , sklearn.preprocessing.StandardScaler
                    ]

# now that we have defined the columns and the classes of transformers we can use gen_features
# in order to generate a list of tuples suitable for DataFrameMapper

feature_def = gen_features(
    columns = col_categorical
    , classes = classes_categorical
)

feature_def_numerical = gen_features(
    columns = col_numerical
    , classes = classes_numerical
)

feature_def.extend(feature_def_numerical)

# when constructing the mapper we can specify whether we want a dataframe or a numpy array as output

mapper_df = DataFrameMapper( feature_def , df_out = True )

mapper_np = DataFrameMapper( feature_def , df_out = False )

mapped_df = mapper_df.fit_transform( df.copy() )

mapped_np = mapper_np.fit_transform( df.copy() )

print( mapped_np[1:10,1:20] )

mapped_df.head(10)




[[1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0.]]


,sex,embarked_C,embarked_Q,embarked_S,class_First,class_Second,class_Third,who_child,who_man,who_woman,...,deck_G,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton,alone,pclass,age,sibsp,parch,fare
0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.827377,-0.565736,0.432793,-0.473674,-0.502445
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,-1.566107,0.663861,0.432793,-0.473674,0.786845
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.827377,-0.258337,-0.474545,-0.473674,-0.488854
3,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,-1.566107,0.433312,0.432793,-0.473674,0.420730
4,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.827377,0.433312,-0.474545,-0.473674,-0.486337
5,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.827377,-0.104637,-0.474545,-0.473674,-0.478116
6,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,-1.566107,1.893459,-0.474545,-0.473674,0.395814
7,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.827377,-2.102733,2.247470,0.767630,-0.224083
8,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.827377,-0.181487,-0.474545,2.008933,-0.424256
9,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,-0.369365,-1.180535,0.432793,-0.473674,-0.042956


the results are looking really good, its almost as good as `recipes`. However if we wanted to apply a boxcox transformation on top of it we would have to write our own `scikit-learn` like transformer. However the transformer will be added in a future version so I would not bother with that at the moment.

## `feather`
The package `feather-format` uses `Apache arrow` to efficiently save dataframes on disk. It is compatible with many other programming languages including `R`

In [16]:
import feather
import os

if not os.path.isdir('./data'):
    os.mkdir('./data')


df_feather = mapped_df.\
    assign( y = y )

feather.write_dataframe(df_feather, './data/mapped_df.feather')


## Sparcity ratio

In [12]:
print('sparcity ratio original data:', round( sparsity_ratio(X), 2) )
print('sparcity ratio tranformed data:', round( sparsity_ratio(mapped_np), 2) )

sparcity ratio original data: 0.17
sparcity ratio tranformed data: 0.56


The transformation have resulted in a matrix with a high sparcity thus we will test whether we might benefit from converting to a sparse matrix format

In [74]:
from scipy import sparse
from time import time
from sklearn.tree import DecisionTreeClassifier


X_sparse = sparse.coo_matrix(mapped_np)

clf_sparse = DecisionTreeClassifier()
clf_dense = DecisionTreeClassifier()

t0 = time()
clf_sparse.fit(X_sparse, y)
print('exec time sparse:', round( time() - t0,3 ) )


t0 = time()
clf_dense.fit(mapped_np, y)
print('exec time dense :', round( time() - t0,3 ) )


exec time sparse: 0.012
exec time dense : 0.008


We can see that our decision tree classifiert does not benefit from  a sparse data format.

# Pipelines

Pipelines are constructs that chain scikit preprocessing steps together and attaching an optional classifier or a regressor to the end. 

We can then use the pipe as we would use a regular model we can fit it and get predictions, we could get crossvalidated performance scores or perform parameter tuning. This has a couple of advantages.

- The code becomes more compact and readable
- Instead of saving multiple transformers (scaling, boxcox ) we can simply store one to apply to future data
- We can tune several steps of the pipeline in one go (for example feature selector + model tuning parameters)

We are going to contruct two pipes one for preprocessing and one for model fitting. It makes sense to seperate these two because we the first one contains a defined sequence of steps and the last pipe we are going to use to tune certain parameters via cross validation. 

When performning the cross validation the transformers and estimators in the pipe will be applied **after** splitting the data into cross validation pairs. Here it makes only sense to include steps that need to be unbiased such as feature selection and modelling algorithms. 

## Preprocessing Pipeline

We are going to apply the `sklearn-pandas` dataframe mapper and a low variance feature filter.

In [80]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import VarianceThreshold
from scipy import stats
import os


pipe_pre_process = sklearn.pipeline.Pipeline([
    ('mapper', mapper_np ) 
    , ('feats', VarianceThreshold() )
])


pipe_pre_process

Pipeline(memory=None,
     steps=[('mapper', DataFrameMapper(default=False, df_out=False,
        features=[(['sex'], [CategoricalImputer(copy=True, missing_values='NaN'), LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)]), (['embarked'], [CategoricalImputer(copy=True, missing_values='NaN'), LabelBinarizer(neg_la...h_std=True)])],
        input_df=False, sparse=False)), ('feats', VarianceThreshold(threshold=0.0))])

In [81]:
pipe_pre_process.named_steps

{'feats': VarianceThreshold(threshold=0.0),
 'mapper': DataFrameMapper(default=False, df_out=False,
         features=[(['sex'], [CategoricalImputer(copy=True, missing_values='NaN'), LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)]), (['embarked'], [CategoricalImputer(copy=True, missing_values='NaN'), LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)]), (['class'], [CategoricalImp...es='NaN', strategy='median', verbose=0), StandardScaler(copy=True, with_mean=True, with_std=True)])],
         input_df=False, sparse=False)}

The parameters are saved as follows in a nested dictionary and are named after the following principle `step_name + '__' + argument`

In [82]:
pipe_pre_process.get_params()

{'feats': VarianceThreshold(threshold=0.0),
 'feats__threshold': 0.0,
 'mapper': DataFrameMapper(default=False, df_out=False,
         features=[(['sex'], [CategoricalImputer(copy=True, missing_values='NaN'), LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)]), (['embarked'], [CategoricalImputer(copy=True, missing_values='NaN'), LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)]), (['class'], [CategoricalImp...es='NaN', strategy='median', verbose=0), StandardScaler(copy=True, with_mean=True, with_std=True)])],
         input_df=False, sparse=False),
 'mapper__default': False,
 'mapper__df_out': False,
 'mapper__features': [(['sex'],
   [CategoricalImputer(copy=True, missing_values='NaN'),
    LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)]),
  (['embarked'],
   [CategoricalImputer(copy=True, missing_values='NaN'),
    LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)]),
  (['class'],
   [CategoricalImputer(copy=True, missing_values='Na

We can set a parameter

In [83]:
pipe_pre_process.set_params(feats__threshold = 0.05)
pipe_pre_process.named_steps.feats

VarianceThreshold(threshold=0.05)

Then we fit the preprocessing pipe to the data

In [84]:
pipe_pre_process.fit(X)
X_proc = pipe_pre_process.fit_transform(X)

X_proc

array([[ 1.        ,  0.        ,  0.        , ...,  0.43279337,
        -0.47367361, -0.50244517],
       [ 0.        ,  1.        ,  0.        , ...,  0.43279337,
        -0.47367361,  0.78684529],
       [ 0.        ,  0.        ,  0.        , ..., -0.4745452 ,
        -0.47367361, -0.48885426],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.43279337,
         2.00893337, -0.17626324],
       [ 1.        ,  1.        ,  0.        , ..., -0.4745452 ,
        -0.47367361, -0.04438104],
       [ 1.        ,  0.        ,  1.        , ..., -0.4745452 ,
        -0.47367361, -0.49237783]])

## Modelling Pipeline

We will add a feature selection step, which choses variables based on a univariate test such as a chisquare test (which we cannot use here because it does not accept negative values) and ANOVA and then fit a decision tree.

In [86]:
pipe_mod = sklearn.pipeline.Pipeline([
    ('feats', sklearn.feature_selection.SelectKBest( k = 10) ) 
    , ('tree', sklearn.tree.DecisionTreeClassifier() )
])


We can apply the same '__' synthax as we used for setting the parameters of the pipe for constructing the dictionary for the sandomized hyperparameter search

In [89]:

param_dist = dict( tree__min_samples_split = stats.randint(2,250)
                 , tree__min_samples_leaf = stats.randint(1,500)
                 , tree__min_impurity_decrease = stats.uniform(0,1)
                 , tree__max_features = stats.uniform(0,1)
                 , feats__score_func = [sklearn.feature_selection.f_classif ## Anova
                                       , sklearn.feature_selection.mutual_info_classif] ) ## nearest n

n_iter = 500

random_search = RandomizedSearchCV(pipe_mod
                                   , param_dist
                                   , n_iter = n_iter
                                   , scoring = 'roc_auc'
                                   , cv = RepeatedKFold( n_splits = 5, n_repeats = 3 )
                                   , verbose = True
                                   , n_jobs = 4 ## parallel processing
                                   , return_train_score = True
                                  )


random_search.fit(X = X_proc, y =  df.survived )



Fitting 30 folds for each of 500 candidates, totalling 15000 fits


[Parallel(n_jobs=4)]: Done 165 tasks      | elapsed:   10.7s
[Parallel(n_jobs=4)]: Done 1408 tasks      | elapsed:  1.2min
[Parallel(n_jobs=4)]: Done 3047 tasks      | elapsed:  2.9min
[Parallel(n_jobs=4)]: Done 4847 tasks      | elapsed:  5.3min
[Parallel(n_jobs=4)]: Done 7393 tasks      | elapsed:  8.1min
[Parallel(n_jobs=4)]: Done 10732 tasks      | elapsed: 11.6min
[Parallel(n_jobs=4)]: Done 14406 tasks      | elapsed: 16.0min
[Parallel(n_jobs=4)]: Done 15000 out of 15000 | elapsed: 16.7min finished


RandomizedSearchCV(cv=<sklearn.model_selection._split.RepeatedKFold object at 0x000001FFFEEABB00>,
          error_score='raise',
          estimator=Pipeline(memory=None,
     steps=[('feats', SelectKBest(k=10, score_func=<function f_classif at 0x000001FFFCC336A8>)), ('tree', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))]),
          fit_params=None, iid=True, n_iter=500, n_jobs=4,
          param_distributions={'tree__min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001FFFEEABBA8>, 'tree__min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001FFFEEAB0F0>, 'tree__min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_froz

In [90]:
random_search.best_estimator_

Pipeline(memory=None,
     steps=[('feats', SelectKBest(k=10,
      score_func=<function mutual_info_classif at 0x000001FFFCC59048>)), ('tree', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=0.700962458667292, max_leaf_nodes=None,
            min_impurity_decrease=0.003543609591600383,
            min_impurity_split=None, min_samples_leaf=27,
            min_samples_split=13, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'))])